In [30]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [31]:
## Function to map input major names with output column names in dataframe
def degree_dicts(df, columns):
    temp_col = columns[columns['VARIABLE NAME'].str.contains('CIP')]
    var_name = temp_col['VARIABLE NAME']
    var_name = var_name.astype('string')
    
    degs = temp_col['NAME OF DATA ELEMENT'].str.split('Bachelor\'s degree in ', expand=True).iloc[:,1]
    degs = degs.str.split('.', expand=True).iloc[:,0]
    degs = degs.astype('string')
    return dict(zip(degs, var_name))

In [32]:
## Data we're currently using along with column names (to retrieve the right major column)
data = pd.read_csv('../data/final.csv')
norm = pd.read_csv('../data/final_standardized.csv')
columns = pd.read_excel('../data/scorecard/columns-simplified.xlsx')

/home/huy/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
## Dictionary to create query and importance
school_dict = {}

In [34]:
## All variables we'll be getting from front-end
## Anything commented out is a variable I don't know how to handle yet
## No weather, cost, or gpa in any of this code

major = "Engineering"
zipcode = 92804
sat = 1400
act = 30
states = ["CA", "NY"]
size = 2500
environment = 1 ## 1=city, 2=suburb, 3=town, 4=rural
climate = "Bwk" ## states and climate zone?

## Cost calculation from tuition either in-state or out-of-state
## weather -> abhishek
## climate type

In [35]:
## Adding variables to dictionary
school_dict[degree_dicts(data, columns)[major]] = []
school_dict[degree_dicts(data, columns)[major]].append(1)

school_dict['SAT_AVG'] = [] 
school_dict['SAT_AVG'].append(sat)

school_dict['ACTCMMID'] = []
school_dict['ACTCMMID'].append(act)

for state in states:
    school_dict['STABBR.'+state] = []
    school_dict['STABBR.'+state].append(1)
    
school_dict['UGDS'] = []
school_dict['UGDS'].append(size)

school_dict['LOCALE.' + str(environment)] = []
school_dict['LOCALE.' + str(environment)].append(1)

In [36]:
## Selecting Variable Importance
size_imp = 4
environment_imp = 2
states_imp = 3

## For any variable w/o a corresponding importance value, I set importance to avg (3)
school_dict[degree_dicts(data, columns)[major]].append(3)
school_dict['SAT_AVG'].append(3)
school_dict['ACTCMMID'].append(3)

for state in states:
    school_dict['STABBR.' + state].append(states_imp)
    
school_dict['UGDS'].append(size_imp)

school_dict['LOCALE.' + str(environment)].append(environment_imp)

## teach_qual_imp -> teaching quality
## select_imp -> selectivity
## proj_earn_imp -> projected earnings

In [37]:
query = pd.DataFrame(pd.DataFrame(school_dict).iloc[0, :]).T
importance = pd.DataFrame(school_dict).iloc[1, :]

In [38]:
norm = norm[school_dict.keys()]

In [44]:
norm

,CIP14BACHL,SAT_AVG,ACTCMMID,STABBR.CA,STABBR.NY,UGDS,LOCALE.1
0,1.426437,-1.674284,-1.709673,-0.295543,-0.307414,0.120186,0.978108
1,1.426437,1.052445,0.982979,-0.295543,-0.307414,1.166084,0.978108
2,-0.619999,-1.165972,-1.123273,-0.295543,-0.307414,-0.516673,0.978108
3,1.426437,1.838113,1.581346,-0.295543,-0.307414,0.446534,0.978108
4,1.426437,-1.609582,-1.709673,-0.295543,-0.307414,-0.077258,0.978108
...,...,...,...,...,...,...,...
2322,-0.619999,-0.291694,-0.363347,-0.295543,-0.307414,-0.460334,0.978108
2323,-0.619999,-0.486416,-0.570781,-0.295543,-0.307414,-0.549256,0.978108
2324,-0.619999,-0.554014,-0.602694,-0.295543,-0.307414,-0.558213,-1.021942
2325,-0.619999,0.045373,0.019608,-0.295543,-0.307414,-0.524851,-1.021942


In [39]:
## Custom distance function with importance
def custom_dist(x, y, col_imp):
    sum_dist = 0
    for i in range(len(x)):
        sum_dist += col_imp[i]*(x[i] - y[i])**2
    return np.sqrt(sum_dist)

In [40]:
neigh = NearestNeighbors(metric=custom_dist, metric_params = {'col_imp': importance})
neigh.fit(norm)

NearestNeighbors(metric=<function custom_dist at 0x7f33f45ec4c0>,
                 metric_params={'col_imp': CIP14BACHL    3
SAT_AVG       3
ACTCMMID      3
STABBR.CA     3
STABBR.NY     3
UGDS          4
LOCALE.1      2
Name: 1, dtype: int64})

In [43]:
data.loc[neigh.kneighbors(query, 10, return_distance=False)[0], ]

,UNITID,INSTNM,CITY,STABBR,ZIP,INSTURL,MAIN,NUMBRANCH,CONTROL,REGION,...,CLIMATE_ZONE.BWh,CLIMATE_ZONE.BWk,CLIMATE_ZONE.Cfa,CLIMATE_ZONE.Cfb,CLIMATE_ZONE.Csa,CLIMATE_ZONE.Csb,CLIMATE_ZONE.Dfa,CLIMATE_ZONE.Dfb,CLIMATE_ZONE.Dfc,CLIMATE_ZONE.Dwa
2020,433387,Western Governors University,Salt Lake City,UT,84107,www.wgu.edu/,1,1,2,7,...,0,0,0,0,0,0,0,1,0,0
961,183026,Southern New Hampshire University,Manchester,NH,03106-1045,www.snhu.edu/,1,1,2,1,...,0,0,0,0,0,0,0,1,0,0
1478,214777,Pennsylvania State University-Main Campus,University Park,PA,16802-1503,www.psu.edu/,1,23,1,2,...,0,0,0,0,0,0,0,1,0,0
2245,484613,University of Phoenix-Arizona,Phoenix,AZ,85040,https://www.phoenix.edu/,1,16,3,6,...,1,0,0,0,0,0,0,0,0,0
274,132903,University of Central Florida,Orlando,FL,32816,www.ucf.edu/,1,1,1,5,...,0,0,1,0,0,0,0,0,0,0
1687,228723,Texas A & M University-College Station,College Station,TX,77843-1248,https://www.tamu.edu/,1,2,1,6,...,0,0,1,0,0,0,0,0,0,0
40,104717,Grand Canyon University,Phoenix,AZ,85017,www.gcu.edu/,1,1,3,6,...,0,0,0,0,0,0,0,0,0,0
1321,204796,Ohio State University-Main Campus,Columbus,OH,43210,www.osu.edu/,1,6,1,3,...,0,0,1,0,0,0,0,0,0,0
1748,232557,Liberty University,Lynchburg,VA,24515,www.liberty.edu/,1,1,2,5,...,0,0,1,0,0,0,0,0,0,0
37,104151,Arizona State University Campus Immersion,Tempe,AZ,85287,www.asu.edu/,1,13,1,6,...,1,0,0,0,0,0,0,0,0,0
